In [1]:
import libsql_client

import polars as pl
import polars.selectors as cs
import pandas as pd
import os
from dotenv import load_dotenv

import datetime

from utils.maps import CLEANED_BRAND_NAMES, BRAND_PREMIUM_STATUS, BRANDS_COUNTRY_CONTINENT

pl.Config.set_fmt_str_lengths(100)

polars.config.Config

In [2]:
load_dotenv()

True

In [3]:

url = os.environ.get("DB_URL")
auth_token = os.environ.get("DB_AUTH_TOKEN")

if not url or not auth_token:
    raise ValueError("DB_URL or AUTH_TOKEN not found in .env")

client = libsql_client.create_client(
    url=url,
    auth_token=auth_token,
)

In [4]:
results = await client.execute("""
select * from cars
""")

In [5]:
xd = list(zip(*results.rows))

data = {
    col: row for i, (row, col)  in enumerate(zip(xd, results.columns))
}

pldf = pl.DataFrame(data=data)

In [119]:
analytics = (
    pldf
    .with_columns(
        pl.when(pl.col("price").str.contains("Consultar"))
        .then("None")
        .otherwise(pl.col("price"))
        .name.keep())
    .with_columns([
        pl.when(pl.col(pl.Utf8) == "None")
        .then(None)
        .otherwise(pl.col(pl.Utf8))
        .name.keep()
    ])
    .unique(subset=["link"])
    .filter(pl.col("title") != "Sin Titulo Encontrado")
    .filter(pl.col("brand")!="Otros")
    .filter(pl.col("price").is_not_null())
    .filter(pl.col("kilometers").is_not_null())
    .with_columns([
        pl.col("price").str.split("$").list.last().str.replace(",", "").cast(pl.Float32),
        pl.col("cc").str.replace(",", "").str.split(" ").list.first().str.slice(0, 4).cast(pl.Int32),
        pl.col("kilometers").str.replace(",", "").str.split(" ").list.first().str.replace(",", "").cast(pl.Int32),
        pl.col("title").str.split(" ").list.last().alias("year_of_manufacture").cast(pl.Int32),
        pl.when(pl.col("brand").is_in(CLEANED_BRAND_NAMES.keys())).then(pl.col("brand").map_dict(CLEANED_BRAND_NAMES)).otherwise(pl.col("brand")).alias("brand")
    ])
    .filter(pl.col("price") > 1) # some prices are 1, which is not possible
    .with_columns(
        pl.when(pl.col("brand").is_in(BRAND_PREMIUM_STATUS.keys())).then(pl.col("brand").map_dict(BRAND_PREMIUM_STATUS)).otherwise(None).alias("brand_status"),
        pl.when(pl.col("brand").is_in(BRANDS_COUNTRY_CONTINENT.keys())).then(pl.col("brand").map_dict(BRANDS_COUNTRY_CONTINENT)).otherwise(None).alias("brand_struct"),
        pl.col("year_of_manufacture").cut([1950, 1980, 2000, 2009, 2014, 2019]).cast(pl.Utf8).alias("period_of_manufacture"),
        pl.col("kilometers").cut([10_000, 50_000, 100_000, 180_000]).cast(pl.Utf8).alias("kilometers"),
    )
    # treat null values in kilometers and cc
    .unnest("brand_struct")
    .sort("parsed_at", descending=True)
)

/var/folders/2g/hjzhlsnn11n_54l_z9c74h2c0000gn/T/ipykernel_25897/2264928152.py:5: DeprecationWarning: in a future version, string input will be parsed as a column name rather than a string literal. To silence this warning, pass the input as an expression instead: `pl.lit('None')`
  .then("None")


# Clustering

In [136]:
cols_to_passthrough = ["id", "parsed_at"]
cols_to_passthrough_test_2 = ["id", "link", "parsed_at", "price", "title", "model", "version"]
colst_to_ohe = analytics.select(pl.exclude(cols_to_passthrough_test_2)).select(pl.col(pl.Utf8)).columns
cols_to_standarize = analytics.select(pl.exclude(cols_to_passthrough_test_2)).select(cs.numeric()).columns

In [149]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.cluster import HDBSCAN
from sklearn.impute import KNNImputer

# make a pipeline that numeric columns are standardScalered and str are OH encoded, "id", "parsed_at" are passthrough
pipeline = make_pipeline(
    ColumnTransformer(
        [
            # ("passthrough", "passthrough", cols_to_passthrough),
            ("knn_imputer", KNNImputer(), ["cc"]),
            ("standard_scaler", MinMaxScaler(), cols_to_standarize),
            ("one_hot_encoder", OneHotEncoder(), colst_to_ohe)
        ]
    ),
    HDBSCAN()
)

clusters = pipeline.fit_predict(analytics.to_pandas())

In [138]:
# ohe = analytics.select(pl.exclude(cols_to_passthrough_test_2)).select(pl.col(pl.Utf8)).fill_null("").to_dummies()
# numeric_cols = analytics.select(pl.exclude(cols_to_passthrough_test_2)).select(cs.numeric()).columns
# numeric = analytics.select((pl.col(col)-pl.mean(col))/pl.std(col) for col in numeric_cols)


# xddf = pl.concat([ohe, numeric], how="horizontal").to_pandas()

In [164]:
format = "%d/%m/%y %H:%M:%S"

(
    analytics
    .with_columns(pl.Series("cluster", clusters))
    .filter(pl.col("cluster") > 0)
    .with_columns(((datetime.datetime.now() - pl.col("parsed_at").str.strptime(pl.Datetime, format)).dt.total_hours() < 24).alias("is_today"))
    .with_columns(
                  pl.col("price").mean().over('cluster').alias("cluster_mean").cast(pl.Int32),
                  pl.col("price").median().over('cluster').alias("cluster_median").cast(pl.Int32),
                  pl.col("price").std().over('cluster').alias("cluster_std").cast(pl.Int32),
    )
    .with_columns(((pl.col("price") < pl.col("cluster_median")*0.4)).alias("is_cheap"))
    .filter(pl.col("cluster") == 251)
    .sort("is_cheap", descending=True)
    # .filter(pl.col("is_cheap") & pl.col("is_today"))
    # .get_column("cluster")
    # .unique()
    # .to_list()
)

id,parsed_at,link,title,model,price,kilometers,cc,fuel_type,transmission_type,category,brand,version,year_of_manufacture,brand_status,country,continent,period_of_manufacture,cluster,is_today,cluster_mean,cluster_median,cluster_std,is_cheap
i64,str,str,str,str,f32,str,i32,str,str,str,str,str,i32,str,str,str,str,i32,bool,i32,i32,i32,bool
7100,"""25/11/23 18:04:03""","""https://www.neoauto.com/auto/usado/nissan-sentra-2017-1758439""","""Nissan Sentra 2017""","""Sentra""",4400.0,"""(10000, 50000]""",1800,"""Gasolina""","""Mecánica""","""Sedan""","""Nissan""",null,2017,"""Normal Brand""","""Japan""","""Asia""","""(2014, 2019]""",251,true,11981,12310,3426,true
6959,"""25/11/23 18:01:44""","""https://www.neoauto.com/auto/usado/nissan-sentra-2016-1758543""","""Nissan Sentra 2016""","""Sentra""",9700.0,"""(10000, 50000]""",1800,"""Gasolina""","""Automática""","""Sedan""","""Nissan""",null,2016,"""Normal Brand""","""Japan""","""Asia""","""(2014, 2019]""",251,true,11981,12310,3426,false
6650,"""22/11/23 18:53:53""","""https://www.neoauto.com/auto/usado/nissan-sentra-2018-1757939""","""Nissan Sentra 2018""","""Sentra""",14490.0,"""(10000, 50000]""",1799,"""Gasolina""","""Automática""","""Sedan""","""Nissan""",null,2018,"""Normal Brand""","""Japan""","""Asia""","""(2014, 2019]""",251,false,11981,12310,3426,false
6533,"""19/11/23 13:19:23""","""https://www.neoauto.com/auto/usado/nissan-sentra-2016-1751257""","""Nissan Sentra 2016""","""Sentra""",10500.0,"""(100000, 180000]""",1800,"""Gasolina""","""Mecánica""","""Sedan""","""Nissan""",null,2016,"""Normal Brand""","""Japan""","""Asia""","""(2014, 2019]""",251,false,11981,12310,3426,false
6106,"""19/11/23 13:12:17""","""https://www.neoauto.com/auto/usado/nissan-sentra-2017-1752119""","""Nissan Sentra 2017""","""Sentra""",13200.0,"""(10000, 50000]""",1800,"""Gasolina""","""Automática""","""Sedan""","""Nissan""",null,2017,"""Normal Brand""","""Japan""","""Asia""","""(2014, 2019]""",251,false,11981,12310,3426,false
4378,"""19/11/23 12:43:34""","""https://www.neoauto.com/auto/usado/nissan-sentra-2016-1755346""","""Nissan Sentra 2016""","""Sentra""",12300.0,"""(10000, 50000]""",1800,"""Gasolina""","""Automática""","""Sedan""","""Nissan""",null,2016,"""Normal Brand""","""Japan""","""Asia""","""(2014, 2019]""",251,false,11981,12310,3426,false
1663,"""19/11/23 11:58:21""","""https://www.neoauto.com/auto/usado/nissan-sentra-2015-1717295""","""Nissan Sentra 2015""","""Sentra""",11900.0,"""(10000, 50000]""",1800,"""Gasolina""","""Automática""","""Sedan""","""Nissan""",null,2015,"""Normal Brand""","""Japan""","""Asia""","""(2014, 2019]""",251,false,11981,12310,3426,false
1014,"""19/11/23 11:47:19""","""https://www.neoauto.com/auto/usado/nissan-sentra-2018-1753953""","""Nissan Sentra 2018""","""Sentra""",17500.0,"""(10000, 50000]""",1800,"""Gasolina""","""Automática""","""Sedan""","""Nissan""",null,2018,"""Normal Brand""","""Japan""","""Asia""","""(2014, 2019]""",251,false,11981,12310,3426,false
353,"""19/11/23 11:36:02""","""https://www.neoauto.com/auto/usado/nissan-sentra-2016-1749018""","""Nissan Sentra 2016""","""Sentra""",12320.0,"""(10000, 50000]""",1800,"""Gasolina""","""Automática""","""Sedan""","""Nissan""",null,2016,"""Normal Brand""","""Japan""","""Asia""","""(2014, 2019]""",251,false,11981,12310,3426,false


In [ ]:
[17, 20, 49, 131, 251]